In [2]:
import pandas as pd 
import networkx as nx 
import os 
import os.path as osp 

In [3]:
# from graphein.protein.config import ProteinGraphConfig

# config = ProteinGraphConfig()
# config.dict()

In [4]:
# from graphein.protein.edges.atomic import add_atomic_edges
# params_to_change = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}

# config = ProteinGraphConfig(**params_to_change)
# config.dict()

In [5]:
# params_to_change = {"granularity": "centroids"}

# config = ProteinGraphConfig(**params_to_change)
# config.dict()

In [6]:
# from graphein.protein.graphs import construct_graph

# from graphein.protein.edges.distance import add_hydrogen_bond_interactions, add_peptide_bonds


# g = construct_graph(config=config, pdb_path="/p300s/wangmx_group/xutingfeng/SIS/data/complex_pdb/S11-SLF3:S11-RNase.pdb")



In [7]:
# from graphein.protein.visualisation import plotly_protein_structure_graph

# p = plotly_protein_structure_graph(
#     g,
#     colour_edges_by="kind",
#     colour_nodes_by="element_symbol",
#     label_node_ids=False,
#     node_size_min=5,
#     node_alpha=0.85,
#     node_size_multiplier=1,
#     plot_title="Atom-level graph (PDB: 3eiy). Nodes coloured by their Element"
#     )
# p.show()


In [8]:
def extract_file_name(absolute_path):
    filename_with_suffix = osp.split(absolute_path)[-1]
    filename = osp.splitext(filename_with_suffix)[0]
    return filename




In [26]:
from graphein.protein.visualisation import plotly_protein_structure_graph
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.visualisation import plotly_protein_structure_graph

from graphein.protein.graphs import construct_graph,construct_graphs_mp
from graphein.protein.edges.distance import (add_peptide_bonds,
                                             add_hydrogen_bond_interactions,
                                             add_disulfide_interactions,
                                             add_ionic_interactions,
                                             add_aromatic_interactions,
                                             add_aromatic_sulphur_interactions,
                                             add_cation_pi_interactions
                                            )

from graphein.protein.features.nodes.amino_acid import meiler_embedding, amino_acid_one_hot, expasy_protein_scale

params_to_change = {
    # "granularity": "centroids",
    "edge_construction_functions": [add_peptide_bonds,
                                                  add_aromatic_interactions,
                                                  add_hydrogen_bond_interactions,
                                                  add_disulfide_interactions,
                                                  add_ionic_interactions,
                                                  add_aromatic_sulphur_interactions,
                                                  add_cation_pi_interactions],
            "node_metadata_functions": [amino_acid_one_hot]
    }

config = ProteinGraphConfig(**params_to_change)
# config.dict()

pdb_root = "../data/complex_pdb/" 
pdb_list =[osp.join(pdb_root, pdb) for pdb in os.listdir(pdb_root) if pdb.endswith(".pdb")]

# graph_dict = {extract_file_name(pdb_path):construct_graph(config=config, pdb_path = pdb_path) for pdb_path in pdb_list }
graph_dict = construct_graphs_mp(pdb_path_it=pdb_list, config=config)
graph_dict = {extract_file_name(k):v for k,v in graph_dict.items()}

g = graph_dict["S11-SLF3:S11-RNase"]

p = plotly_protein_structure_graph(
    g,
    colour_edges_by="kind",
    colour_nodes_by="degree",
    label_node_ids=False,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1
    )
p.show()

  0%|          | 0/74 [00:00<?, ?it/s]

In [27]:
import torch 
from torch_geometric.data import Data 

In [28]:
# g = construct_graph(pdb_path="/p300s/wangmx_group/xutingfeng/SIS/data/complex_pdb/S11-SLF3:S11-RNase.pdb")

In [29]:
amino_acid_alphabet = {'A': 'ALA',
 'F': 'PHE',
 'C': 'CYS',
 'U': 'SEC',
 'D': 'ASP',
 'N': 'ASN',
 'E': 'GLU',
 'Q': 'GLN',
 'G': 'GLY',
 'H': 'HIS',
 'L': 'LEU',
 'I': 'ILE',
 'K': 'LYS',
 'O': 'PYL',
 'M': 'MET',
 'P': 'PRO',
 'R': 'ARG',
 'S': 'SER',
 'T': 'THR',
 'V': 'VAL',
 'W': 'TRP',
 'Y': 'TYR'}

In [30]:
# prot_prot = pd.read_csv("../data/protprot.csv", sep=",")
# prot_prot

In [31]:
# complex_list = set([extract_file_name(i) for i in os.listdir("../data/fasta/complex")])

# protprot_list= set([i+":"+j for i,j in list(prot_prot.set_index(["chain_A_name","chain_B_name"]).index.unique())])


In [32]:
# complex_list.difference(protprot_list)

In [33]:
# protprot_list.difference(complex_list)

In [34]:
import pandas as pd 
from functools import partial

prot_prot = pd.read_csv("../data/protprot.csv", sep=",")

def prot_prot_nodes_turn(prot_prot_df, graph_dict):
    
    graph_name = prot_prot_df["chain_A_name"].values[0]+":"+prot_prot_df["chain_B_name"].values[0]
    print(graph_name)
    graph = graph_dict[graph_name]

    seq_A = graph.graph["sequence_A"]
    seq_B = graph.graph["sequence_B"]
    
    edges_list = []
    nodes_list = []
    for idx, series in prot_prot_df.iterrows():

        aa_pos_A = series["chain_A_resName"]
        aa_A = amino_acid_alphabet[seq_A[aa_pos_A-1]]
        node_A = f"A:{aa_A}:{aa_pos_A}"

        aa_pos_B = series["chain_B_resName"]
        aa_B = amino_acid_alphabet[seq_B[aa_pos_B-1]]
        node_B = f"B:{aa_B}:{aa_pos_B}"

        kind = series["interaction_type"]
        distance = series["distance"]

        edges_list.append((node_A, node_B, kind, distance))
        nodes_list.append(node_A)
        nodes_list.append(node_B)

    return pd.Series({"nodes":set(nodes_list), "edges":edges_list})

map_func = partial(prot_prot_nodes_turn, graph_dict=graph_dict)

prot_prot = prot_prot.groupby(["chain_A_name", "chain_B_name"]).apply(map_func)
prot_prot

S11-SLF1:S11-RNase
S11-SLF1:S17-RNase
S11-SLF1:S19-RNase
S11-SLF1:S5-RNase
S11-SLF1:S7-RNase
S11-SLF1:S9-RNase
S11-SLF2:S11-RNase
S11-SLF2:S17-RNase
S11-SLF2:S19-RNase
S11-SLF2:S5-RNase
S11-SLF2:S7-RNase
S11-SLF2:S9-RNase
S11-SLF3:S11-RNase
S11-SLF3:S17-RNase
S11-SLF3:S19-RNase
S11-SLF3:S5-RNase
S11-SLF3:S7-RNase
S11-SLF3:S9-RNase
S11-SLF4:S11-RNase
S11-SLF5:S11-RNase
S17-SLF2:S17-RNase
S17-SLF3:S17-RNase
S17-SLF4:S17-RNase
S17-SLF5:S17-RNase
S17-SLF6:S17-RNase
S5-SLF1:S11-RNase
S5-SLF1:S17-RNase
S5-SLF1:S19-RNase
S5-SLF1:S5-RNase
S5-SLF1:S7-RNase
S5-SLF1:S9-RNase
S5-SLF2:S11-RNase
S5-SLF2:S17-RNase
S5-SLF2:S19-RNase
S5-SLF2:S5-RNase
S5-SLF2:S7-RNase
S5-SLF2:S9-RNase
S5-SLF3:S5-RNase
S5-SLF4:S5-RNase
S5-SLF5:S5-RNase
S5-SLF6:S5-RNase
S7-SLF1:S11-RNase
S7-SLF1:S17-RNase
S7-SLF1:S19-RNase
S7-SLF1:S5-RNase
S7-SLF1:S7-RNase
S7-SLF1:S9-RNase
S7-SLF2:S11-RNase
S7-SLF2:S17-RNase
S7-SLF2:S19-RNase
S7-SLF2:S5-RNase
S7-SLF2:S7-RNase
S7-SLF2:S9-RNase
S7-SLF3:S11-RNase
S7-SLF3:S17-RNase
S7-SLF3:S1

nodes  \
chain_A_name chain_B_name                                                      
S11-SLF1     S11-RNase     {B:TRP:56, B:ASN:170, A:PRO:186, A:PHE:359, B:...   
             S17-RNase     {A:SER:154, A:PRO:186, B:LYS:41, A:GLU:176, A:...   
             S19-RNase     {B:THR:6, A:PHE:359, B:LYS:58, A:ILE:285, A:HI...   
             S5-RNase      {A:SER:154, B:TRP:56, A:PHE:359, B:LYS:41, A:I...   
             S7-RNase      {B:ARG:91, A:SER:154, A:PRO:186, B:LYS:41, A:G...   
...                                                                      ...   
S9-SLF17     S9-RNase      {A:ARG:192, B:GLU:24, B:TYR:78, A:GLY:106, B:H...   
S9-SLF3      S9-RNase      {A:TYR:177, A:GLU:175, B:LYS:73, B:GLU:24, B:T...   
S9-SLF4      S9-RNase      {A:LYS:77, A:PHE:228, A:ASP:250, B:GLU:24, A:T...   
S9-SLF5      S9-RNase      {B:LYS:73, A:LEU:182, B:GLN:89, A:GLN:239, B:H...   
S9-SLF9      S9-RNase      {B:LYS:73, B:GLU:24, A:TYR:175, B:LEU:72, A:CY...   

                                                                       edges  
chain_A_name chain_B_name                                                     
S11-SLF1     S11-RNase     [(A:LYS:68, B:GLU:157, Hydrogen bonds, 2.61), ...  
             S17-RNase     [(A:SER:154, B:LYS:41, Hydrogen bonds, 2.75), ...  
             S19-RNase     [(A:GLU:181, B:LYS:15, Hydrogen bonds, 2.64), ...  
             S5-RNase      [(A:LYS:68, B:GLU:215, Hydrogen bonds, 3.2), (...  
             S7-RNase      [(A:GLU:181, B:LYS:41, Hydrogen bonds, 2.61), ...  
...                                                                      ...  
S9-SLF17     S9-RNase      [(A:GLU:183, B:TYR:47, Hydrogen bonds, 2.7), (...  
S9-SLF3      S9-RNase      [(A:SER:102, B:GLU:76, Hydrogen bonds, 2.85), ...  
S9-SLF4      S9-RNase      [(A:LYS:77, B:VAL:158, Hydrogen bonds, 2.65), ...  
S9-SLF5      S9-RNase      [(A:CYS:186, B:LYS:73, Hydrogen bonds, 2.57), ...  
S9-SLF9      S9-RNase      [(A:GLY:147, B:HIS:22, Hydrogen bonds, 2.75), ...  

[74 rows x 2 columns]

In [35]:
from copy import deepcopy

chain_A_name = "S11-SLF1"
chain_B_name = "S11-RNase"
complex_name = chain_A_name + ":" + chain_B_name

g = deepcopy(graph_dict[complex_name])
g_without = deepcopy(graph_dict[complex_name])

added_edge_list = prot_prot.loc[chain_A_name, chain_B_name]["edges"]
for edge in added_edge_list:
    g.add_edge(edge[0], edge[1], kind = {edge[2]}, distance = edge[3])


sub_graph_nodes = prot_prot.loc[chain_A_name, chain_B_name]["nodes"]
test = g.subgraph(sub_graph_nodes)
g_without = g_without.subgraph(sub_graph_nodes)


In [36]:
test = g.subgraph(sub_graph_nodes)
test 

In [37]:
# prot_prot_dict = prot_prot.loc["S11-SLF3","S11-RNase"]["prot_prot"]

# sub_graph_node = []

# for node in list(g.nodes):
#     chain, aa, pos = node.split(":")
#     if int(pos) in prot_prot_dict[chain]:
#         sub_graph_node.append(node)
        
# sub_graph_node
    

In [38]:
p = plotly_protein_structure_graph(
    g_without,
    colour_edges_by="kind",
    colour_nodes_by="degree",
    label_node_ids=True,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1
    )
p.show()

In [60]:
import matplotlib.pyplot as plt 

import matplotlib.colors as mcolors 
colors = ['grey', 'blue']
colour_map = mcolors.LinearSegmentedColormap.from_list("test", colors)

p = plotly_protein_structure_graph(
    test,
    colour_edges_by="kind",
    colour_nodes_by="chain",
    label_node_ids=True,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1,
    node_colour_map = colour_map
    )
p.show()

In [61]:
import matplotlib.colors as mcolors 
colors = ['darkorange', 'gold', 'lawngreen', 'lightseagreen']
colour_map = mcolors.ListedColormap(colors)


G = test

colors = ['darkorange', 'gold', 'lawngreen', 'lightseagreen']
colour_map = mcolors.ListedColormap(colors)

chains = G.graph["chain_ids"]
# print(chains)
chain_colours = dict(
    zip(chains, list(colour_map(1 / len(chains), 1, len(chains))))
)
print(chain_colours)
colors = [chain_colours[d["chain_id"]] for n, d in G.nodes(data=True)]

{'A': 124, 'B': 252}


AttributeError: 'LinearSegmentedColormap' object has no attribute 'color'

In [49]:
test.add_edge("A:ASN:214", "'A:TYR:213'")

NetworkXError: Frozen graph can't be modified

In [72]:
test = nx.Graph(test)

test.add_edge("A:ASN:214", "A:TYR:213", weight=set([3]))

In [73]:
test.edges(data=True)

EdgeDataView([('A:TYR:213', 'A:ASN:214', {'kind': {'peptide_bond'}, 'distance': 3.7903609326817413, 'weight': {3}}), ('A:TYR:213', 'A:PHE:215', {'kind': {'aromatic'}, 'distance': 5.830936631451245}), ('B:ARG:202', 'B:ARG:201', {'kind': {'peptide_bond'}, 'distance': 3.8459710087310848}), ('B:ARG:202', 'A:ASP:280', {'kind': {'ionic'}, 'distance': 12.761793682707772}), ('B:ARG:202', 'A:GLU:261', {'kind': {'ionic'}, 'distance': 9.471625995572248}), ('A:GLU:261', 'A:ARG:283', {'kind': {'ionic'}, 'distance': 7.283369137974541}), ('A:GLU:261', 'A:ARG:282', {'kind': {'ionic'}, 'distance': 9.120565388176328}), ('A:PHE:151', 'A:CYS:150', {'kind': {'peptide_bond'}, 'distance': 3.824037918222045}), ('A:PRO:281', 'A:ASP:280', {'kind': {'peptide_bond'}, 'distance': 3.8630248510720198}), ('A:PRO:281', 'A:ARG:282', {'kind': {'peptide_bond'}, 'distance': 3.8874203528818434}), ('A:ARG:283', 'A:ARG:282', {'kind': {'peptide_bond'}, 'distance': 3.8316420761861347}), ('A:ARG:283', 'A:ASP:280', {'kind': {'io